## Imports

In [2]:
import re
import json
from tqdm import tqdm
from bs4 import BeautifulSoup
import bs4
import os
import sys

In [3]:
# Python has a recursion limit of 3K.
# We raise it up to 10K
sys.setrecursionlimit(10000)

In [4]:
DIR_RAW_DATA_PATH = './data/endata/'
DIR_CLEAN_DATA = './data/endata_new_clean/'
JSON_TRAIN_CORPUS = './data/wiki_html_all.json'

## MAIN

In [22]:
# Open Train Corpus file
with open(JSON_TRAIN_CORPUS,'w')as g:
    for root, dirs, files in os.walk(DIR_RAW_DATA_PATH):

        iter_i = 0
        # Iter ZERO
        for dir in dirs:
            #print("DIR: "+dir)
            sub_DIR_RAW_DATA_PATH = os.path.join(root,dir)
            # Get folder
            iter_j=0
            # Iter ONE
            for sub_root,sub_dirs,sub_files in os.walk(sub_DIR_RAW_DATA_PATH):
                if iter_j > 0:
                    extracted_name = sub_root.split("\\")[1].split("(")[0]
                    #print("FOLDER: "+extracted_name)
                iter_j += 1
                # Get file
                iter_k = 0
                # Iter TWO
                for item_parsed in sub_files:
                    path = os.path.join(sub_root,item_parsed)
                    path = re.sub(r'\\', '/', path)
                    clean_path = os.path.join(DIR_CLEAN_DATA,dir+extracted_name+"/clean_"+item_parsed)
                    clean_path = re.sub(r'\((\d+)\)', '-', clean_path)
                    print(f"PATH: {path}")
                    print(f"FILE: {clean_path}")
                    print("")
                    if iter_k == 0:
                        break
                    iter_k += 1

                if iter_j == 2:
                    break
            iter_i =+1 

                

        """ 
        for dir in tqdm(dirs):
            sub_DIR_RAW_DATA_PATH = os.path.join(root,dir)
            for sub_root,sub_dirs,sub_files in os.walk(sub_DIR_RAW_DATA_PATH):
                # foeach file in dir
                for item_parsed in tqdm(sub_files):
                    #index = item_parsed.split('.')[0]
                    path = os.path.join(sub_root,item_parsed)
                    clean_path = DIR_CLEAN_DATA+'_clean.html'
                    cleaner = HTMLCleaner(path,clean_path)
                    cleaner.clean_html()
                    #cleaner.clean_tree()
                    storer = HTMLStorer(input_file=None,html=cleaner.line)
                    g.write(json.dumps(storer.data,ensure_ascii=False)+'\n') """


PATH: ./data/endata/auto/auto-aol(2000)/0000.htm
FILE: ./data/endata_new_clean/autoauto-aol/clean_0000.htm

PATH: ./data/endata/book/book-abebooks(2000)/0000.htm
FILE: ./data/endata_new_clean/bookbook-abebooks/clean_0000.htm

PATH: ./data/endata/camera/camera-amazon(1767)/0000.htm
FILE: ./data/endata_new_clean/cameracamera-amazon/clean_0000.htm

PATH: ./data/endata/job/job-careerbuilder(2000)/0000.htm
FILE: ./data/endata_new_clean/jobjob-careerbuilder/clean_0000.htm

PATH: ./data/endata/movie/movie-allmovie(2000)/0000.htm
FILE: ./data/endata_new_clean/moviemovie-allmovie/clean_0000.htm

PATH: ./data/endata/nbaplayer/nbaplayer-espn(434)/0000.htm
FILE: ./data/endata_new_clean/nbaplayernbaplayer-espn/clean_0000.htm

PATH: ./data/endata/restaurant/restaurant-fodors(2000)/0000.htm
FILE: ./data/endata_new_clean/restaurantrestaurant-fodors/clean_0000.htm

PATH: ./data/endata/university/university-collegeboard(2000)/0000.htm
FILE: ./data/endata_new_clean/universityuniversity-collegeboard/clean

In [31]:
class HTMLCleaner(object):
    def __init__(self, input_file, output_file):
        self.input_file = input_file
        self.output_file = output_file
        self.line = ""
        
    # CLEAN HTML String --------------------------------------- start
    def clean_tag(self, x):
        '''Split the matched tag into components
        Check if the first component starts with '<'
        If it does, reconstruct the tag with the first and second components
        If not, reconstruct the tag with the first component
        Return the cleaned tag'''

        x = x.group().split()
        if x[0] == '<': 
            y = x[0] + x[1].strip('>') + '>'
        else:
            y = x[0].strip('>') + '>'
        return y
    

    def clean_html(self):
        '''Preprocess an HTML string and cleans it

        Define regular expressions for different HTML elements
        Compile regular expressions into patterns
        Initialize lists to store different elements

        Open file
        - Extract annotations
        - Extract close tags
        - Extract styles
        - Extract all tags

        Store the cleaned line'''

        style_regex = "(?:<style.*?>(?:.|[\r\n])*?</style>|<script.*?>(?:.|[\r\n])*?</script>)"
        all_tag_regex = "(?:<(?:!|/?[a-zA-Z]+).*?/?>)"
        close_tag_regex = '(<(?:!|/?[a-zA-Z]+)[^>]*?/>){1}?'
        annotation_regex = '(?:<!--(?:.|[\r\n])*?-->)'
        close_tag_pattern = re.compile(close_tag_regex)
        annotation_pattern = re.compile(annotation_regex)
        all_tag_pattern = re.compile(all_tag_regex)
        style_pattern = re.compile(style_regex)
        tags = []
        annotation = []
        close_tag = []
        all_tag = []
        style = []
        with open(self.input_file, 'r') as f:
            line = f.read()
            annotation.extend(annotation_pattern.findall(line))
            #print(annotation)
            line = re.sub(annotation_regex, '', line)
            close_tag.extend(close_tag_pattern.findall(line))
            #print(close_tag)
            line = re.sub(close_tag_regex, '', line)
            style.extend(style_pattern.findall(line))
            #print(style)
            line = re.sub(style_regex, '', line)
            all_tag.extend(all_tag_pattern.findall(line))
            #print(all_tag)
            line = re.sub(all_tag_regex, self.clean_tag, line)  # Note: self.clean_tag is missing
        #print(line)
        self.line = line
    # CLEAN HTML String --------------------------------------- end



    # BUILD THREE --------------------------------------- start
    def if_text(self, node):
        '''
        Check if the text content of the HTML or XML node is empty
        Return 0 if the text is empty (considered "falsy")
        Return 1 if the text is not empty (considered "truthy")'''
        if node.get_text('|', strip=True) == "":
            return 0
        else:
            return 1
    
    def get_tree(self,root):
        '''
        Recursively processes the HTML/XML tree rooted at 'root', removing elements with empty text content.

        Args: root (bs4.element.Tag): The root of the HTML/XML tree.

        Returns: bs4.element.Tag: The modified root after removing elements with empty text content.
        '''
        delete_list = []
        # Iterate through the children of the root
        for child in root.children:
            # Check if the child is a bs4.element.Tag
            if type(child) == bs4.element.Tag:
                print(child)
                print("-----------")
                # Check if the text content of the tag is empty
                if not self.if_text(child):
                    delete_list.append(child)
                else:
                    # Recursively process child if it's an HTML tag
                    child = self.get_tree(child)
            elif type(child) == bs4.element.NavigableString:
                # Check if the NavigableString (text) is empty
                if str(child).strip() == "":
                    delete_list.append(child)
        # Remove items in the delete_list from the tree
        for item in delete_list:
            print(item.extract())
        # Return the modified root
        return root
    
    def merge_tree(self,root,k):
        i = 0
        while(i<len(root.contents)):
            if(type(root.contents[i])==bs4.element.Tag):
                if len(root.contents[i].contents)<=k:
                    tmp = root.contents[i]
                    j = i
                    del root.contents[i]
                    for item in tmp.contents:
                        root.contents.insert(j,item)
                        j+=1
                else:
                    root.contents[i] = self.merge_tree(root.contents[i],k)
                    i+=1
            else:
                i+=1
        return root
                    
    def clean_tree(self):
        root = BeautifulSoup(self.line,'html.parser').html
        root = self.get_tree(root)
        root = self.merge_tree(root,2)
        self.root = root
    
    # BUILD THREE --------------------------------------- end


    def store(self):
        with open(self.output_file,'w+')as f:
            f.write(str(self.root))

m_path = "./data/endata/auto/auto-aol(2000)/0000.htm"
m_clean_path = "./data/endata_new_clean/autoauto-aol/clean_0000.htm"
cleaner = HTMLCleaner(m_path, m_clean_path)
cleaner.clean_html()
cleaner.clean_tree()

<head>
<title>2010 Hyundai Accent Blue 2dr Hatchback Overview</title>
</head>
-----------
<title>2010 Hyundai Accent Blue 2dr Hatchback Overview</title>
-----------
<body>
<div>
<div><div></div></div>
<div><div><ul><li><a>AOL</a></li><li><a>MAIL</a></li><li><ul><li>You might also like:</li><li><a> Autoblog</a>,</li><li><a> Autoblog Green</a></li><li> and <a>More</a></li></ul></li></ul><ul></ul></div><div><div><a>AOL Autos</a></div><div><form><div></div></form></div></div><ul><li><a>Main</a></li><li><a><span>New Cars</span></a><ul><li><a>Research Cars</a></li><li><a>Car Finder</a></li><li><a>Find Inventory</a></li><li><a>AOL Autos Best Deals</a></li><li><a>Dealer Directory</a></li><li><a>Compare Cars</a></li><li><a>Owner Reviews</a></li><li><a>Available Incentives</a></li><li><a>Car Loans</a></li></ul></li><li><a><span>Used Cars</span></a><ul><li><a>Used Car Search</a></li><li><a>Certified Pre-Owned</a></li><li><a>Kelley Blue Book Values</a></li><li><a>Vehicle History Reports</a></li><l